In [1]:
import requests
from bs4 import BeautifulSoup
import time
import random
import pandas as pd

In [2]:
import sys
import os

In [3]:
def flushprint(www):
    sys.stdout.write('\r')
    sys.stdout.write('%s' %www)
    sys.stdout.flush()

In [4]:
# 2-城市-酒店详情页面（评分、评分等级、评论人数）.ipynb 文件保存的数据
hotel_list=pd.read_excel('beijing_score.xls',index_col=0)

### 拼接并存储酒店详情页面的url

In [8]:
label=hotel_list.iloc[2]['link'].split('/hotel/cn/')[1].split('.zh-cn.html?')[1].split('&sid=')[0]#随电脑不变

In [9]:
#酒店的评论页的url，以便之后跳转该页面翻页
#label是访问booking自带的识别信息，num为酒店数量
def review_url(label,num):
    review_url_list=[]
    label=label
    for index in range(0,num):
        pagename_=hotel_list.iloc[index]['link'].split('/hotel/cn/')[1].split('.zh-cn.html?')[0]
        review_url='https://www.booking.com/reviewlist.zh-cn.html?'+label+'&pagename='+pagename_+'&cc1=cn'
        review_url_list.append(review_url)
    return review_url_list

In [10]:
hotel_list['review-url']=review_url(label,790)

In [11]:
hotel_list['review-content']='NaN'

### 计算评论页面翻页范围
#### 整理review-num列的数据格式（1，830不识别，1830才可以）

In [12]:
#判断评论翻页的范围
#num为评论数量（review-num获得）
def page_offset(num):
    offset_range=0
    if int(num)%10==0:
        offset_range=int(num)
    else:
        offset_range=(int(num)//10+1)*10
    return offset_range

In [13]:
from collections import defaultdict

In [14]:
hotel_review_list=[]
hotel_review_error=[]

In [15]:
for index in range(0,790):
    review_num=hotel_list.iloc[index]['review-num']
    review_url=hotel_list.iloc[index]['review-url']
    #判断是否有评论人数,有则翻页爬取
    if review_num==review_num:
        offset_range=page_offset(review_num)#计算翻页的offset范围
        review_content=defaultdict(int)
        try:   
            for page in range(0,offset_range,10):  
                page_url=review_url+'&offset='+str(page)+'&rows=10'
                tt = random.random()
                flushprint(str(page_url)+':sleeping'+str(tt)+'seconds')
                time.sleep(tt)

                html=requests.get(page_url,headers=headers)
                html.encoding='utf-8'
                content=html.text
                soup=BeautifulSoup(content,'html.parser')
                
                for item in soup.find_all('span',class_='bui-avatar-block__subtitle'):
                    user_country=item.text.replace('\n\n','')
                    if  user_country in review_content:
                        review_content[user_country]+=1
                    else:
                        review_content[user_country]=1
            hotel_review_list.append(review_content)
        
        except Exception as e:
            print(index,review_num,review_url,e)
            error_review=[index,review_num,review_url]
            hotel_review_error.append(error_review)
            continue
    else:
        review_='NaN'
        hotel_review_list.append(review_)

https://www.booking.com/reviewlist.zh-cn.html?label=gen173nr-1DCAQoggJCDXNlYXJjaF_ljJfkuqxIK1gEaDGIAQGYASu4AQfIAQzYAQPoAQH4AQOIAgGoAgO4AqShzu8FwAIB&pagename=bu-ke-qi-yuan-hostel&cc1=cn&offset=80&rows=10:sleeping0.28625601848037374secondsecondsssdsondseconds3secondseping0.755213090601925secondsdsss

In [17]:
hotel_list['review-content']=hotel_review_list

In [18]:
hotel_list.to_excel('beijing_review.xls')